In [1]:
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('../01_preprocess/preprocessed2.csv')
# Calculate the percentile brackets
df['percentile bracket'] = pd.qcut(df['time'], q=10, labels=False, duplicates='drop') + 1

df.drop(['bin_name','cycle', 'instr', 'LLC_miss','CPU','L1_miss', 'br_miss', 'scale', 'counted_final_key_count', 'GHz', 'time', 'task', 'IPC'], axis=1, inplace=True)
df.head()

,config_name,const_basicHintCount,const_enableBasicHead,const_enableDense,const_enableDense2,const_enableDensifySplit,const_enableHash,const_enableHashAdapt,const_enableHeadNode,const_enablePrefix,...,ycsb_range_len,ycsb_zipf,nodeCount_Leaf,nodeCount_Inner,nodeCount_Dense,nodeCount_Hash,nodeCount_Head4,nodeCount_Head8,nodeCount_Dense2,percentile bracket
0,hash,16,1,0,0,0,1,0,0,1,...,100,1.133614,0,846,0,121433,0,0,0,5
1,hash,16,1,0,0,0,1,0,0,1,...,100,1.133614,0,846,0,121433,0,0,0,4
2,dense3,16,1,1,0,1,0,0,0,1,...,100,0.316120,38703,270,0,0,0,0,0,3
3,dense3,16,1,1,0,1,0,0,0,1,...,100,0.316120,38703,270,0,0,0,0,0,8
4,hints,16,1,0,0,0,0,0,0,1,...,100,0.666485,27850,742,0,0,0,0,0,6


In [2]:
# Identify non-numeric columns
non_numeric_columns = df.select_dtypes(['object']).columns

print("Number of columns:", df.shape[1])

# Apply one-hot encoding to non-numeric columns
df = pd.get_dummies(df, columns=non_numeric_columns)

print("Number of columns after one-hot:", df.shape[1])

Number of columns: 35
Number of columns after one-hot: 40


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('percentile bracket', axis=1), df['percentile bracket'], test_size=0.2, random_state=42)

# Create an instance of the RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Fit the classifier to the training data
rf_classifier.fit(X_train, y_train)

# Predict the output variable for the testing data
y_pred = rf_classifier.predict(X_test)

# Evaluate the performance of the classifier
accuracy = accuracy_score(y_test, y_pred)
classification_report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:")
print(classification_report)


Accuracy: 0.8740031897926634
Classification Report:
              precision    recall  f1-score   support

           1       0.89      0.99      0.94       113
           2       0.90      0.83      0.86       113
           3       0.91      0.84      0.87       123
           4       0.86      0.81      0.84       118
           5       0.91      0.91      0.91       127
           6       0.85      0.91      0.88       128
           7       0.80      0.80      0.80       121
           8       0.80      0.82      0.81       153
           9       0.87      0.87      0.87       129
          10       0.99      0.98      0.98       129

    accuracy                           0.87      1254
   macro avg       0.88      0.87      0.87      1254
weighted avg       0.87      0.87      0.87      1254



In [8]:
from sklearn import tree
decision_tree = rf_classifier.estimators_[9]
df['percentile bracket'] = df['percentile bracket'].astype(str)+"0%"
text_representation = tree.export_graphviz(decision_tree, feature_names=df.drop('percentile bracket', axis=1).columns, class_names=df['percentile bracket'].unique(), filled=True, out_file="tree99.dot")
